# IMPORT LIBRARIES

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import json
import os

# LOAD DATA

### JSON DATA

In [2]:
SP_FILE_PATH = 'C:\PROJECTS\emboendo\dicom_viewer\_static\Jsons\points_fix_d.json'

if os.path.exists(SP_FILE_PATH ):
        
        with open(SP_FILE_PATH , 'r') as file:
            json_d=json.load(file)
            display(list(json_d.keys()))
        
else:print(' File does not exist')

['I0', 'I1040', 'I1060', 'I107', 'I213', 'I302', 'I349', 'I434', 'I70', 'I996']

### GENERATE IMPUT_PATHS_SORTED

In [3]:
input_P_S={rec:{}for rec in json_d.keys()}

for R,frms in json_d.items():

    for ii in range(len(json_d[R].keys())):
        input_P_S[R][ii]=frms[str(ii)]["cropped_image"]

input_P_S

{'I0': {0: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_0.npy',
  1: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_1.npy',
  2: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_2.npy',
  3: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_3.npy',
  4: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_4.npy',
  5: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_5.npy',
  6: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_6.npy',
  7: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_7.npy',
  8: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_8.npy',
  9: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_9.npy',
  10: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_10.npy',
  11: 'C:/PROJECTS/emboendo/Data/np_data_files/square_fixed/cropped\\I0_11.npy',
  12: 'C:/PROJECTS/emboendo/Data/np_data_f

### GENERATE VIDEO DICT

In [12]:
video_d={rec:{}for rec in json_d.keys()}

for R,frms in json_d.items():

    for ii in range(len(json_d[R].keys())):
        video_d[R][ii]=np.load(frms[str(ii)]["cropped_image"])

for R in video_d:
    print(R,list(video_d[R].keys()))


I0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
I1040 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
I1060 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
I107 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
I213 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
I302 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
I349 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
I434 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
I70 [0, 1, 2, 3,

In [13]:
video_d={rec:[] for rec in json_d.keys()}

for R,frms in json_d.items():

    for ii in range(len(json_d[R].keys())):
        video_d[R].append(np.load(frms[str(ii)]["cropped_image"]))

video_d={rec:np.concatenate(img_ls, axis=0) for rec, img_ls in video_d.items() }
print(np.shape(video_d['I70']),type((video_d['I70'])))

(1800, 100) <class 'numpy.ndarray'>
